# Variables of households and population of Mexican Municipalities in 2020

This Notebook uses the households and population dataframe of Mexican Municipalities (admin2) derived from the 2020 Mexican Census: [INEGI](https://inegi.org.mx/programas/ccpv/2020/#Datos_abiertos).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from string import ascii_letters
import numpy as np

%matplotlib inline
%reload_ext autoreload
%autoreload 2

Read Variables of households and population of Mexico in 2020

In [2]:
df = pd.read_parquet('../data/conjunto_de_datos_iter_00CSV20.parquet')

By using this query only the totals of each variable for each municipality is used well the rest of the dataframe is ignored.

In [3]:
df.query("NOM_LOC == 'Total del Municipio'", inplace = True)
df

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
6,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,None,None,None,948990,...,251719,178619,130290,80951,56131,3299,7293,74227,731,*
574,1,Aguascalientes,2,Asientos,0,Total del Municipio,None,None,None,51536,...,10682,4526,3882,596,556,377,1447,7206,177,*
749,1,Aguascalientes,3,Calvillo,0,Total del Municipio,None,None,None,58250,...,13666,6553,4749,1382,1352,443,904,8061,138,*
918,1,Aguascalientes,4,Cosío,0,Total del Municipio,None,None,None,17000,...,3424,1741,1664,223,191,139,414,1912,50,*
985,1,Aguascalientes,5,Jesús María,0,Total del Municipio,None,None,None,129929,...,31408,19920,13483,9296,6582,480,1150,11447,148,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195372,32,Zacatecas,54,Villa Hidalgo,0,Total del Municipio,None,None,None,19446,...,3929,1150,1090,109,106,160,744,3569,80,*
195422,32,Zacatecas,55,Villanueva,0,Total del Municipio,None,None,None,31558,...,7418,4411,5181,643,569,306,746,4252,101,*
195542,32,Zacatecas,56,Zacatecas,0,Total del Municipio,None,None,None,149607,...,39755,31989,28630,11944,7639,681,1185,8783,129,*
195616,32,Zacatecas,57,Trancoso,0,Total del Municipio,None,None,None,20455,...,3834,1092,1854,204,185,130,677,3274,59,*


By using the dictonary that the dataset offers, the selection of columns of interest is done.

In [4]:
df=df[['ENTIDAD','NOM_ENT','MUN','NOM_MUN','PCON_DISC','PCON_LIMI','PCLIM_PMEN','PSIND_LIM','GRAPROES','PSINDER','PDER_SS','PROM_OCUP',
       'TVIVPARHAB','VPH_SINTIC','POB0_14','POB15_64','POB65_MAS']].copy()

Based on the dictonary the columns are renamed in a clearer way.

In [5]:
df.rename(columns = {'MUN':'municipality_number', 'NOM_MUN': 'municipalities','PCON_DISC': 'population_disability','PCON_LIMI': 'population_limitation',
                     'PCLIM_PMEN': 'population_mental_problem','PSIND_LIM':'population_no_problems','GRAPROES': 'average_years_finish', 'PSINDER': 'no_med_insurance', 
                     'PDER_SS': 'med_insurance', 'PROM_OCUP': 'average_household_size','TVIVPARHAB': 'total_households','VPH_SINTIC': 'household_no_tics',
                    'POB0_14':'population_0_14_years_old','POB15_64':'population_15_64_years_old','POB65_MAS':'population_65_more_years_old'}, inplace=True)

To properly merge the dataframe from the week 1 analyzes with the dataframe that is currently being analyze it is necessary to obtain the code that describes the state of origin of the municipality.

In [6]:
df['mun_num'] = df['municipality_number'].apply(lambda i: f'{i:03d}')
df['ENTIDAD'] = df['ENTIDAD'].astype(str)
df['cve_ent'] = df['ENTIDAD'] + df['mun_num']

It is also necessary to change the data types of the columns of interest to int and float data types, since this values will be normalized for further study.

In [7]:
df['population_disability'] = df['population_disability'].astype(int)
df['population_limitation'] = df['population_limitation'].astype(int)
df['population_mental_problem'] = df['population_mental_problem'].astype(int)
df['population_no_problems'] = df['population_no_problems'].astype(int)
df['average_years_finish'] = df['average_years_finish'].astype(float)
df['no_med_insurance'] = df['no_med_insurance'].astype(int)
df['med_insurance'] = df['med_insurance'].astype(int)
df['average_household_size'] = df['average_household_size'].astype(float)
df['total_households'] = df['total_households'].astype(int)
df['household_no_tics'] = df['household_no_tics'].astype(int)
df['population_0_14_years_old'] = df['population_0_14_years_old'].astype(int)
df['population_15_64_years_old'] = df['population_15_64_years_old'].astype(int)
df['population_65_more_years_old'] = df['population_65_more_years_old'].astype(int)

To obtain the total household which have TIC's it is necessary to substract from the total household the households that do not have TIC's

In [8]:
df['household_tics'] = df['total_households']-df['household_no_tics']

The week 1 analyzes it is read

In [9]:
dfWeek1 = pd.read_csv('../data/week1analyzesMunicipalities.csv')

The week 1 analyzes cve_ent is converted to a string value for a good compatibility for future merging

In [10]:
dfWeek1['cve_ent'] = dfWeek1['cve_ent'].astype('str')
dfWeek1.head()

,cve_ent,municipality,population,total_cases,case_rate,total_cases_last_60_days,case_rate_last_60_days,total_deaths,death_rate,total_deaths_last_60_days,death_rate_last_60_days
0,31001,Abala,7035,66,938.166311,11,156.361052,10,142.146411,0,0.000000
1,5001,Abasolo,1130,8,707.964602,1,88.495575,1,88.495575,0,0.000000
2,19001,Abasolo,2918,29,993.831391,1,34.270048,3,102.810144,0,0.000000
3,11001,Abasolo,95581,720,753.287787,17,17.785962,92,96.253439,1,1.046233
4,28001,Abasolo,12768,80,626.566416,35,274.122807,4,31.328321,0,0.000000


The week 1 analyzes and the lastest dataframe is merged using the code of the state of origin of the municipality.

In [11]:
dfAll = pd.merge(df,dfWeek1,on=['cve_ent'])
dfAll.head()

,ENTIDAD,NOM_ENT,municipality_number,municipalities,population_disability,population_limitation,population_mental_problem,population_no_problems,average_years_finish,no_med_insurance,...,municipality,population,total_cases,case_rate,total_cases_last_60_days,case_rate_last_60_days,total_deaths,death_rate,total_deaths_last_60_days,death_rate_last_60_days
0,1,Aguascalientes,1,Aguascalientes,47525,116705,13945,777394,10.84,173199,...,Aguascalientes,961977,23801,2474.175578,1744,181.293316,2143,222.770399,72,7.484586
1,1,Aguascalientes,2,Asientos,2886,4551,692,43707,8.54,7590,...,Asientos,50864,473,929.930796,9,17.694243,36,70.776974,1,1.966027
2,1,Aguascalientes,3,Calvillo,4088,6760,1191,46834,8.05,11832,...,Calvillo,60760,1022,1682.027650,40,65.832785,26,42.791310,0,0.000000
3,1,Aguascalientes,4,Cosío,805,1933,206,14166,9.08,2347,...,Cosio,16918,133,786.144934,4,23.643457,12,70.930370,0,0.000000
4,1,Aguascalientes,5,Jesús María,5036,11904,1642,112021,10.22,27299,...,Jesus Maria,130184,837,642.936152,58,44.552326,62,47.624900,1,0.768144


Once merged the dataframes only the data that is possible to normalized is selected. After selecting the data the normalization of it is implemented based on the total population or total households of each municipality by obtain the percentage of people or households with the certain variable of interest.

In [12]:
dfAll = dfAll[['cve_ent','municipality','population','population_disability', 'population_limitation',
       'population_mental_problem','population_no_problems', 'average_years_finish', 'no_med_insurance',
       'med_insurance', 'average_household_size', 'case_rate', 
       'case_rate_last_60_days', 'death_rate',
       'death_rate_last_60_days','total_households','household_tics','household_no_tics',
        'population_0_14_years_old','population_15_64_years_old','population_65_more_years_old']].copy()
dfAll['pct_disability']=dfAll['population_disability']/dfAll['population']*100
dfAll['pct_limitation']=dfAll['population_limitation']/dfAll['population']*100
dfAll['pct_mental_problem']=dfAll['population_mental_problem']/dfAll['population']*100
dfAll['pct_no_problems']=dfAll['population_no_problems']/dfAll['population']*100
dfAll['pct_no_med_insurance']=dfAll['no_med_insurance']/dfAll['population']*100
dfAll['pct_med_insurance']=dfAll['med_insurance']/dfAll['population']*100
dfAll['pct_household_tics']=dfAll['household_tics']/dfAll['total_households']*100
dfAll['pct_household_no_tics']=dfAll['household_no_tics']/dfAll['total_households']*100
dfAll['pct_pop_0_14_years_old']=dfAll['population_0_14_years_old']/dfAll['population']*100
dfAll['pct_pop_15_64_years_old']=dfAll['population_15_64_years_old']/dfAll['population']*100
dfAll['pct_pop_65_more_years_old']=dfAll['population_65_more_years_old']/dfAll['population']*100

Finally the variables and the region codes are selected of the dataframe for future storage

In [13]:
dfFinal = dfAll[['cve_ent','municipality','case_rate','case_rate_last_60_days', 'death_rate',
        'death_rate_last_60_days','population','pct_disability',
        'pct_limitation','pct_mental_problem', 'pct_no_problems' ,'average_years_finish',
        'pct_no_med_insurance','pct_med_insurance', 'average_household_size',
        'pct_household_tics','pct_household_no_tics','pct_pop_0_14_years_old',
        'pct_pop_15_64_years_old','pct_pop_65_more_years_old']].copy()
dfFinal.head()

,cve_ent,municipality,case_rate,case_rate_last_60_days,death_rate,death_rate_last_60_days,population,pct_disability,pct_limitation,pct_mental_problem,pct_no_problems,average_years_finish,pct_no_med_insurance,pct_med_insurance,average_household_size,pct_household_tics,pct_household_no_tics,pct_pop_0_14_years_old,pct_pop_15_64_years_old,pct_pop_65_more_years_old
0,1001,Aguascalientes,2474.175578,181.293316,222.770399,7.484586,961977,4.940347,12.131787,1.449619,80.812119,10.84,18.004485,80.497039,3.54,99.725989,0.274011,25.009226,66.481007,7.062643
1,1002,Asientos,929.930796,17.694243,70.776974,1.966027,50864,5.673954,8.947389,1.360491,85.929144,8.54,14.922145,86.351840,4.10,98.588742,1.411258,31.979396,62.753617,6.548836
2,1003,Calvillo,1682.027650,65.832785,42.791310,0.000000,60760,6.728111,11.125741,1.960171,77.080316,8.05,19.473338,76.316656,3.74,99.112312,0.887688,27.518104,59.009217,9.284068
3,1004,Cosio,786.144934,23.643457,70.930370,0.000000,16918,4.758246,11.425700,1.217638,83.733302,9.08,13.872798,86.600071,4.32,98.729997,1.270003,30.636009,63.240336,6.608346
4,1005,Jesus Maria,642.936152,44.552326,47.624900,0.768144,130184,3.868371,9.143981,1.261292,86.048209,10.22,20.969551,78.679408,3.91,99.554405,0.445595,29.422202,65.253026,5.022123


The dataframe is stored

In [14]:
dfFinal.to_csv('../data/week3_variables_municipalities.csv',index=False)